# Motion Intensity Analysis

**<span style="font-size:15px;">Copyright (c) 2024 Sepideh Nikookar</span>**

In [ ]:
import os
import re
import ast
import wfdb
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import scipy.stats as stats
from sklearn import metrics
import scipy.signal as signal
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

### Data Loading

In [ ]:
LIFEBELL_DEV_SENSORS_DIR = '/path/to/Raw/ACC/Data'

# Changing the current working directory
os.chdir(LIFEBELL_DEV_SENSORS_DIR)

# Print the current working directory
print("The Current working directory now is: {0}".format(os.getcwd()))

In [ ]:
# List all files in the directory
files = os.listdir(format(os.getcwd()))

In [ ]:
# Initialize a list to store the data from each record
acc_data = []
acc_fs = 50

# Filter for files starting with "patient_" and ending with ".dat"
acc_files = [file for file in files if file.startswith('RawACCData_') and file.endswith('.dat')]

# Loop through the patient files
for file in acc_files:

    # Extract the patient id from the filename
    record_name = file.split('.')[0]
    id = record_name.split('_')[1]
    
    # Load the record using wfdb.rdsamp()
    acc_signal, header = wfdb.rdsamp(record_name)

    # Convert to DataFrame
    df = pd.DataFrame(acc_signal, columns=['subject_id', 'session_id', 'packet_id', 'time_index', 'x', 'y', 'z', 'label'])
    df['peripheral_id'] = id  # Add the patient ID to the DataFrame

    start_time = int(re.search(r'\d+', header['comments'][1]).group()) 

    # Convert time difference to milliseconds
    time_diff_ms = (1/acc_fs) * 1000  # 1 second = 1000 milliseconds

    # Adjust custom_index by subtracting 1, then convert it to time
    df['inferred_epoch_ms'] = start_time + (df['time_index'] - 1) * time_diff_ms

    # Append the current DataFrame to the list
    acc_data.append(df)

# Combine all the DataFrames into a single DataFrame
raw_acc_data = pd.concat(acc_data, ignore_index=True)

raw_acc_data['inferred_timestamp_utc'] = pd.to_datetime(raw_acc_data.inferred_epoch_ms, unit='ms')

# Sort the data
raw_acc_data = raw_acc_data.sort_values(by=['peripheral_id', 'inferred_epoch_ms'], ascending=[True, True]).reset_index(drop=True)

raw_acc_data = raw_acc_data[[
    'peripheral_id',
    'subject_id', 
    'session_id',
    'packet_id',
    'inferred_epoch_ms', 
    'inferred_timestamp_utc',
    'x', 
    'y', 
    'z', 
    'label']]

In [ ]:
# Map labels and Extract the dictionary part from the string
label_mapping_str = header['comments'][0].split("Label mapping: ")[1]

# Convert the string to a dictionary using ast.literal_eval
label_mapping = ast.literal_eval(label_mapping_str)

# Reverse the dictionary to map numeric values back to string labels
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Map 'Label_value' to string labels using the reversed dictionary
raw_acc_data['label'] = raw_acc_data['label'].map(reverse_label_mapping)

In [ ]:
LIFEBELL_DEV_HR_DIR = '/path/to/HR/Data'

# Changing the current working directory
os.chdir(LIFEBELL_DEV_HR_DIR)

# Print the current working directory
print("The Current working directory now is: {0}".format(os.getcwd()))

In [ ]:
# Initialize a list to store the data from each record
hr_data = []
hr_fs = 1 #Hz


# Filter for files starting with "patient_" and ending with ".dat"
hr_files = [file for file in files if file.startswith('HeartRate_') and file.endswith('.dat')]

# Loop through the patient files
for file in hr_files:

    # Extract the patient id from the filename
    record_name = file.split('.')[0]
    id = record_name.split('_')[1]
    
    # Load the record using wfdb.rdsamp()
    hr_signal, header = wfdb.rdsamp(record_name)

    # Convert to DataFrame
    df = pd.DataFrame(hr_signal, columns=['packet_id', 'time_index', 'heart_rate'])
    df['peripheral_id'] = id  # Add the patient ID to the DataFrame

    start_time = int(re.search(r'\d+', header['comments'][0]).group()) 

    # Convert time difference to milliseconds
    time_diff_ms = (1/hr_fs) * 1000  # 1 second = 1000 milliseconds

    # Adjust custom_index by subtracting 1, then convert it to time
    df['inferred_epoch_ms'] = start_time + (df['time_index'] - 1) * time_diff_ms

    # Append the current DataFrame to the list
    hr_data.append(df)

# Combine all the DataFrames into a single DataFrame
vivalnk_hr = pd.concat(hr_data, ignore_index=True)

vivalnk_hr['patch_timestamp_utc'] = pd.to_datetime(vivalnk_hr.inferred_epoch_ms, unit='ms')

# Sort the data
vivalnk_hr = vivalnk_hr.sort_values(by=['peripheral_id', 'inferred_epoch_ms'], ascending=[True, True]).reset_index(drop=True)


vivalnk_hr = vivalnk_hr[[
    'peripheral_id',
    'packet_id',
    'inferred_epoch_ms', 
    'patch_timestamp_utc',
    'heart_rate']]

### Data Processing

In [ ]:
acc_data = raw_acc_data.copy()

# --- design butterworth bandpass filter and apply it
# filter parameters
sampling_rate=50
nyq = 0.5 * sampling_rate
cutoff_fs=(0.05,2.0)
filter_order=4

nyq_cutoff = [i/nyq for i in cutoff_fs]
filter_type='bandpass'
bwfilter = signal.butter(
    filter_order,
    nyq_cutoff,
    btype=filter_type,
    analog=False,
    output="sos"
)

# sort the data in preparation for filtering
acc_data.sort_values(['peripheral_id', 'inferred_epoch_ms'], inplace=True)

# apply the butterworth filter
acc_data[['x_filt', 'y_filt', 'z_filt']] = np.concatenate(
    acc_data.groupby('peripheral_id')
    .apply(
        lambda x: signal.sosfiltfilt(
            bwfilter, x[['x', 'y', 'z']].values, axis=0, padlen=0
        )
    ).values
)

In [ ]:
# --- calculate magnitude and rolling magnitude values
# Take the absolute value of the filtered axes columns
acc_data[['x_filt','y_filt','z_filt']] = np.abs(acc_data[['x_filt','y_filt','z_filt']])
acc_data['magnitude'] = np.sqrt(acc_data.x_filt**2 + acc_data.y_filt**2 + acc_data.z_filt**2)
acc_data['inferred_timestamp_utc']=pd.to_datetime(acc_data['inferred_timestamp_utc'])

# Sort the DataFrame by 'inferred_timestamp_utc'
acc_data = acc_data.sort_values(by='inferred_timestamp_utc')

# Compute rolling 30s activity count medians for each sample
rolling_magnitude_tmp = acc_data.groupby(['peripheral_id', 'packet_id', 'subject_id', 'session_id', 'label']).rolling(
    f'5s', 
    on='inferred_timestamp_utc', 
    ).median().reset_index(level=0)
    
rolling_magnitude_tmp['rolling_magnitude'] = rolling_magnitude_tmp.magnitude

# Merge the counts columns back with the original activity_data
acc_data = pd.merge(acc_data, rolling_magnitude_tmp[['peripheral_id', 'inferred_timestamp_utc', 'rolling_magnitude']], on=['peripheral_id', 'inferred_timestamp_utc'])

In [ ]:
# Convert inferred_timestamp_utc to microseconds
acc_data['inferred_timestamp_utc'] = acc_data['inferred_timestamp_utc'].astype('datetime64[us]')
vivalnk_hr['patch_timestamp_utc'] = vivalnk_hr['patch_timestamp_utc'].astype('datetime64[us]')

# Sort dataframes by timestamp columns
vivalnk_hr.sort_values('patch_timestamp_utc', inplace=True)
acc_data.sort_values('inferred_timestamp_utc', inplace=True)

# Define columns to keep
hr_data_cols = ['peripheral_id', 'packet_id', 'patch_timestamp_utc', 'heart_rate']
acc_data_cols = ['peripheral_id', 'packet_id',     'subject_id', 'session_id',  'inferred_epoch_ms', 'x', 'y', 'z', 'label', 'inferred_timestamp_utc', 'magnitude', 'rolling_magnitude']

# Perform merge_asof
data = pd.merge_asof(
    vivalnk_hr[hr_data_cols],
    acc_data[acc_data_cols],
    left_on='patch_timestamp_utc',
    right_on='inferred_timestamp_utc',
    by=['peripheral_id', 'packet_id'],
    tolerance=pd.Timedelta('0.15s'),
    direction='nearest'
)

# Filter data to where we have labels
data = data[data.label.notna()]

# Display or further process `data` as needed


In [ ]:
# take the last 20 minutes of each time series, this makes classes more balanced
# only use the last 20 minutes of each time series - this cuts out all of the extra sitting from the respiration protocols
# Group the data by 'id'
grouped = data.groupby('peripheral_id')

# Create an empty dataframe to store the filtered data
filtered_data = pd.DataFrame(columns=data.columns)

# Iterate over each group
for name, group in grouped:
    # Calculate the cutoff time (i.e. 20 minutes ago from the latest timestamp in the group)
    cutoff_time = group['inferred_timestamp_utc'].max() - timedelta(minutes=20)
    
    # Filter the data to only include rows within the last 25 minutes
    filtered_group = group[group['inferred_timestamp_utc'] >= cutoff_time]
    
    # Append the filtered data to the overall dataframe
    filtered_data = pd.concat([filtered_data, filtered_group])

filtered_data['inferred_timestamp_utc'] = pd.to_datetime(filtered_data.inferred_timestamp_utc)
filtered_data['x'] = filtered_data.x.astype(float)
filtered_data['y'] = filtered_data.y.astype(float)
filtered_data['z'] = filtered_data.z.astype(float)
filtered_data['heart_rate'] = filtered_data.heart_rate.astype(int)

# filter out invalid heart rate measurements from firmware
filtered_data = filtered_data[filtered_data.heart_rate > 0]


### Analysis

In [ ]:
# Reset to default Matplotlib style
plt.style.use('default')

plot_data = filtered_data

plt.figure(figsize=(15, 6))
plot_data[~plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(density=True, alpha=0.5, bins=np.arange(0,1,0.005), label='Inactive State', edgecolor='black')
plot_data[plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(density=True, alpha=0.5, bins=np.arange(0,1,0.005), label='Active State', edgecolor='black')


plt.axvline(0.07, c='darkred', label='Proposed Activity Threshold - 0.07')
plt.xlabel('Activity Count (Median over 5s Window)', fontweight='bold', fontsize=14)
plt.ylabel('Time Windows (Count)', fontweight='bold', fontsize=14)
plt.legend()
plt.title('Activity Count Values Separated by Activity \n(Walking, Jogging vs Others)', fontweight='bold', fontsize=14)

# Define x-ticks including 0.07
x_ticks = np.arange(0, 1.1, 0.2)  # Example ticks from 0 to 1 with step of 0.1
x_ticks = np.append(x_ticks, 0.07)  # Add 0.07

# Ensure x_ticks is unique and sorted
x_ticks = np.unique(x_ticks)
x_ticks.sort()

# Update x-ticks
plt.xticks(x_ticks)

# Limit x-axis to 0 and 1
plt.xlim(0, 1)

plt.show()

In [ ]:
active_prediction = plot_data.rolling_magnitude > 0.07
active_label = plot_data.label.isin(['walking', 'jogging'])
print('No Modifications: \n')
print('Accuracy:', round(metrics.accuracy_score(active_label, active_prediction), 2))
print('Balanced Accuracy:', round(metrics.balanced_accuracy_score(active_label, active_prediction), 2))
print('Precision:', round(metrics.precision_score(active_label, active_prediction), 2))
print('Recall:', round(metrics.recall_score(active_label, active_prediction), 2))

### Comparison with cleaned data - no interference, or activity

In [ ]:
plot_data = filtered_data[~filtered_data.label.isin(['interference', 'sitting_activity', 'standing_activity', 'activity', 'transition'])]

plt.figure(figsize=(15, 6))
plot_data[~plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(
    density=True, alpha=0.5, bins=np.arange(0, 1, 0.005), label='Inactive State', edgecolor='black')
plot_data[plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(
    density=True, alpha=0.5, bins=np.arange(0, 1, 0.005), label='Active State', edgecolor='black')

plt.axvline(0.07, c='darkred', label='Proposed Activity Threshold - 0.07')
plt.xlabel('Activity Count (Median over 5s Window)', fontweight='bold', fontsize=14)
plt.ylabel('Time Windows (Count)', fontweight='bold', fontsize=14)
plt.legend()
plt.title('Activity Count Values Separated by Activity \n(Walking, Jogging vs Others)', fontweight='bold', fontsize=14)

# Define x-ticks including 0.07
x_ticks = np.arange(0, 1.1, 0.2)  # Example ticks from 0 to 1 with step of 0.1
x_ticks = np.append(x_ticks, 0.07)  # Add 0.07

# Ensure x_ticks is unique and sorted
x_ticks = np.unique(x_ticks)
x_ticks.sort()

# Update x-ticks
plt.xticks(x_ticks)

# Limit x-axis to 0 and 1
plt.xlim(0, 1)

plt.savefig('Active_vs_Inactive.png', transparent=True)

plt.show()


In [ ]:
active_prediction_refined = plot_data.rolling_magnitude > 0.07
active_label_refined = plot_data.label.isin(['walking', 'jogging'])
print('Accuracy:', round(metrics.accuracy_score(active_label_refined, active_prediction_refined), 2))
print('Balanced Accuracy:', round(metrics.balanced_accuracy_score(active_label_refined, active_prediction_refined), 2))
print('Precision:', round(metrics.precision_score(active_label_refined, active_prediction_refined), 2))
print('Recall:', round(metrics.recall_score(active_label_refined, active_prediction_refined), 2))

### ROC Curve

In [ ]:
# Compute ROC curve and ROC area for initial model
fpr_initial, tpr_initial, _ = roc_curve(active_label, active_prediction)
roc_auc_initial = auc(fpr_initial, tpr_initial)

# Compute ROC curve and ROC area for refined model
# Replace active_prediction_refined and active_label_refined with your refined model predictions
fpr_refined, tpr_refined, _ = roc_curve(active_label_refined, active_prediction_refined)
roc_auc_refined = auc(fpr_refined, tpr_refined)

# Plot ROC curve
plt.style.use('default')
plt.figure(figsize=(8, 6))
plt.plot(fpr_initial, tpr_initial, color='blue', lw=2, label=f'Initial Model (AUC = {roc_auc_initial:.2f})')
plt.plot(fpr_refined, tpr_refined, color='green', lw=2, label=f'Refined Model (AUC = {roc_auc_refined:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontweight='bold', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontweight='bold', fontsize=14)
plt.title('ROC Curve', fontweight='bold', fontsize=16)
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('ROC.png', transparent=True)
plt.show()


### Confusion Matrix

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(active_label_refined, active_prediction_refined)

row_sums = cm.sum(axis=1, keepdims=True)
normalized_cm = cm / row_sums.astype(float)

cm = np.round(normalized_cm, decimals=2)

# Define class labels
classes = ['Active', 'Inactive']

# Convert confusion matrix values to percentages
cm_percent = cm / cm.sum(axis=1, keepdims=True) * 100

# Create the heatmap
plt.figure(figsize=(7, 6))
cmap = plt.get_cmap('Blues')
norm = Normalize(vmin=cm_percent.min(), vmax=cm_percent.max())
ax = sns.heatmap(cm_percent, cmap=cmap, cbar=True,
                 xticklabels=classes, yticklabels=classes, 
                 annot=False)  # Disable default annotations

# Add custom annotations with percentage formatting
for i in range(cm_percent.shape[0]):
    for j in range(cm_percent.shape[1]):
        value = cm_percent[i, j]
        text_color = 'black' if value < 50 else 'white'  # Choose text color based on value
        ax.text(j + 0.5, i + 0.5, f'{value:.1f}%', 
                ha='center', va='center', color=text_color, 
                fontsize=14)

# Customize labels and title
plt.xlabel('Predicted label', fontsize=14, fontweight='bold')
plt.ylabel('Actual Label', fontsize=14, fontweight='bold')
plt.title('Cumulative Confusion Matrix', fontsize=14, fontweight='bold')
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=classes, yticklabels=classes)
# plt.xlabel('Predicted Labels', fontweight='bold', fontsize=14)
# plt.ylabel('True Labels', fontweight='bold', fontsize=14)
plt.savefig('cm(active_vs_inactive).png', transparent=True)
plt.show()

### Subject-Level Views:

In [ ]:
# for peripheral_id in filtered_data.peripheral_id.unique():
#     plot_data = filtered_data[~filtered_data.label.isin(['interference', 'sitting_activity', 'standing_activity', 'activity', 'transition'])]
#     plot_data = filtered_data[filtered_data.peripheral_id == peripheral_id]
#     fig, axes = plt.subplots(figsize=(10,3))
#     plot_data[~plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(density=True, alpha=0.5, bins=np.arange(0,1,0.005), label='Inactive State', edgecolor='black')
#     plot_data[plot_data.label.isin(['walking', 'jogging'])].rolling_magnitude.hist(density=True, alpha=0.5, bins=np.arange(0,1,0.005), label='Active State', edgecolor='black')


#     plt.axvline(0.07, c='darkred', label='Proposed Activity Threshold - 0.07')
#     plt.xlabel('Activity Count  (Median over 5s window)', fontweight='bold', fontsize=14)
#     plt.ylabel('Time Windows (count)', fontweight='bold', fontsize=14)
#     plt.legend()
#     plt.title(f'Activity Count Values Separated by Activity \n(Walking, Jogging vs All Other Postures, activity, interference, and transitions removed)\nID:{peripheral_id}')
#     plt.xlim(0,0.7)
#     plt.ylim(0,50)

### Activity-Level Views:

In [ ]:
# Reset to default Matplotlib style
plt.style.use('default')

for label in filtered_data.label.unique():
    
    plot_data = filtered_data[filtered_data.label == label]
    fig, axes = plt.subplots(figsize=(10,3))
    plot_data.magnitude.hist(density=True, alpha=0.5, bins=np.arange(0,1,0.005), edgecolor='black')


    plt.axvline(0.07, c='darkred', label='Proposed Activity Threshold - 0.07')
    plt.xlabel('Activity Count  (Median over 5s window)', fontweight='bold', fontsize=10)
    plt.ylabel('Time Windows (count)', fontweight='bold', fontsize=10)
    plt.legend()
    plt.title(f'Activity Count Values:{label}')
    plt.xlim(0,1)
    plt.ylim(0,50)

### Vitals View at a Subject-Level through Motion Protocol:

In [ ]:
for peripheral_id in filtered_data.peripheral_id.unique():
    plot_data = filtered_data[(filtered_data.peripheral_id == peripheral_id) & (filtered_data.heart_rate > 0)].copy()
    plot_data.loc[:, 'active'] = plot_data['rolling_magnitude'] > 0.07


    fig, axes = plt.subplots(figsize=(15,5))
    sns.scatterplot(
        x=plot_data.patch_timestamp_utc, 
        y=plot_data.heart_rate,
        hue=plot_data.label,
        s=20,
        style=plot_data.active
        )

    plt.xlabel('Patch Time (UTC)', fontweight='bold', fontsize=10)
    plt.ylabel('Heart Rate', fontweight='bold', fontsize=10)
    plt.legend()
    plt.title(f'Heart Rate Over Motion Protocol \n ID: {peripheral_id}')
    plt.close('all')


### Calculating Resting Vitals

In [ ]:
walk_time = filtered_data[filtered_data.label=='walking'].groupby('peripheral_id').patch_timestamp_utc.min().reset_index()

In [ ]:
resting_data = []
for group, data in filtered_data.groupby('peripheral_id'):
    walk_time = data[data.label == 'walking'].patch_timestamp_utc.min()
    pre_walk_data = data[data.patch_timestamp_utc < walk_time]
    resting_data.append(pre_walk_data[pre_walk_data.label.isin(['lying', 'sitting'])])
resting_data = pd.concat(resting_data)
resting_data['heart_rate'] = resting_data.heart_rate.astype(int)

In [ ]:
resting_hr = resting_data.groupby('peripheral_id').heart_rate.median().reset_index()
resting_std = resting_data.groupby('peripheral_id').heart_rate.std().reset_index()
resting_qrt25 = resting_data.groupby('peripheral_id').heart_rate.quantile(0.25).reset_index()
resting_qrt75 = resting_data.groupby('peripheral_id').heart_rate.quantile(0.75).reset_index()
filtered_data = filtered_data.merge(resting_hr, on='peripheral_id', suffixes=('','_resting'))
filtered_data = filtered_data.merge(resting_std, on='peripheral_id', suffixes=('','_std'))
filtered_data = filtered_data.merge(resting_qrt25, on='peripheral_id', suffixes=('','_qrt25'))
filtered_data = filtered_data.merge(resting_qrt75, on='peripheral_id', suffixes=('','_qrt75'))

In [ ]:
filtered_data['heart_rate_diff'] = filtered_data.heart_rate - filtered_data.heart_rate_resting

In [ ]:
# Scatter plot of 3D magnitude vs. heart rate deviation
# Reset to default Matplotlib style
plt.style.use('default')
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data.rolling_magnitude, filtered_data.heart_rate_diff, s=6, alpha=0.6, color='royalblue')
plt.xlabel('Activity Count (5s Rolling Median)', fontweight='bold', fontsize=14)
plt.ylabel('Deviation from Resting Heart Rate (bpm)', fontweight='bold', fontsize=14)
# plt.title('Relationship between Activity Count and Heart Rate Deviation')
plt.grid(True)
plt.tight_layout()
plt.savefig('activity_vs_heart_rate_deviation.png')
plt.show()

In [ ]:
correlation_coefficient, p_value = stats.pearsonr(filtered_data.rolling_magnitude, filtered_data.heart_rate_diff)

print(f'Pearson correlation coefficient: {correlation_coefficient}')
print(f'P-value: {p_value}')

# Check significance
alpha = 0.05  # significance level
if p_value < alpha:
    print('The correlation is statistically significant.')
else:
    print('The correlation is not statistically significant.')

In [ ]:
for label in filtered_data.label.unique():
    plt.subplots(figsize=(10,3))
    filtered_data[filtered_data.label == label].heart_rate_diff.hist(density=True, bins=range(-30,100,2))
    plt.title(f"Deviation From Resting Heart Rate For Activity: {label}")
    plt.xlabel('Deviation from Resting Heart Rate (in bpm)')
    plt.ylabel('Density (adds to 1)')